In [6]:
from pathlib import Path
import cv2
from PIL import Image

from core.data.services.default_video_service import DefaultVideoService as VideoService
from core.data.services.default_homography_service import DefaultHomographyService as HomographyService
from core.data.services.default_finder_service import DefaultFinderService as FinderService
from core.data.services.default_image_service import DefaultImageService as ImageService
from core.infra.scene_objects.corners import Corners
from core.infra.scene_objects.offset import Offset
from core.infra.context.video_writer_context import VideoWriterContext

In [7]:
assets = Path().cwd().joinpath("assets")
input_video_path = assets.joinpath("videosample_10s.mp4")
minimap_image_path = assets.joinpath("minimap.jpeg")
minimap_video_output_path = Path().cwd().joinpath("output").joinpath("output.mp4")

In [8]:
minimap_image = cv2.imread(str(minimap_image_path))

In [9]:
region_of_interest = Corners.from_rectangle(top_left_x=0, top_left_y=750, bottom_right_x=3840, bottom_right_y=1550)

In [10]:
video_service = VideoService()
homography_service = HomographyService()
finder_service = FinderService()
image_service = ImageService()

In [11]:
minimap_corners = Corners(
    bottom_right=Offset(x=40, y=420),
    bottom_left=Offset(x=40, y=42),
    top_left=Offset(x=752, y=42),
    top_right=Offset(x=752, y=420)
)

In [12]:
scenes = video_service.getScenesFromVideo(video_path=input_video_path, region_of_interest=region_of_interest)

In [13]:
# with VideoWriterContext(path=minimap_video_output_path) as context:
#     for index, (court_scene, court_frame) in enumerate(scenes):
#         court_corners = finder_service.findCourt(court_frame).corners
#         minimap_scene = homography_service.applyHomography(scene=court_scene, from_corners=court_corners, to_corners=minimap_corners)

#         frame = minimap_scene.draw_frame(frame=minimap_image)
        
#         context.write(frame=frame)

#         if index >= 20:
#             break


In [14]:
str(minimap_video_output_path)

'/Users/yann/Documents/Projects/Python/basket-tracking/output/output.mp4'

In [15]:


court_scene, court_frame = next(scenes)
court_corners = finder_service.findCourt(court_frame).corners
minimap_scene = homography_service.applyHomography(scene=court_scene, from_corners=court_corners, to_corners=minimap_corners)

frame1 = minimap_scene.draw_frame(frame=minimap_image)
frame2 = court_scene.draw_frame(frame=court_frame)
Image.fromarray(frame1[:, :, ::-1]).show()
Image.fromarray(frame2[:, :, ::-1]).show()


/Users/yann/Documents/Projects/Python/basket-tracking/.venv/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


### Studying color of clothes

In [16]:
show_frame = lambda frame: Image.fromarray(frame[:, :, ::-1]).show()

In [17]:
show_frame(frame=court_scene.draw_players(frame=court_frame, type='rectangle'))

In [18]:
import numpy as np


copied_frame = court_frame.copy()
for player in court_scene.players_inside_court:
    upper_body: Corners = player.get_upper_body()

    upper_body_frame = court_frame[int(upper_body.top_left.y):int(upper_body.bottom_right.y)-1, int(upper_body.top_left.x):int(upper_body.top_right.x)-1]
    upper_body_frame_hsv = cv2.cvtColor(upper_body_frame, cv2.COLOR_BGR2HSV)

    pixels = np.float32(upper_body_frame_hsv.reshape(-1, 3))

    n_colors = 3

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)

    dominant_hsv = palette[0]
    dominant = cv2.cvtColor(np.uint8([[dominant_hsv]]), cv2.COLOR_HSV2BGR)[0][0]

    
    copied_frame = image_service.drawRectangleInFrame(frame=copied_frame, corners=upper_body, color=tuple(map(int, dominant)))


show_frame(frame=copied_frame)


In [19]:
upper_body = Corners(
    bottom_left=Offset(x=player.corners.bottom_left.x + player_width / 3, y=player.corners.bottom_left.y - player_height / 2),
    bottom_right=Offset(x=player.corners.bottom_right.x - player_width / 3, y=player.corners.bottom_right.y - player_height / 2),
    top_right=Offset(x=player.corners.top_right.x - player_width / 3, y=player.corners.top_right.y + player_height / 8),
    top_left=Offset(x=player.corners.top_left.x + player_width / 3, y=player.corners.top_left.y + player_height / 8),
)

show_frame(frame=image_service.drawRectangleInFrame(frame=court_frame, corners=upper_body))

NameError: name 'player_width' is not defined

In [ ]:
upper_body

In [ ]:
show_frame(frame=image_service.drawRectangleInFrame(frame=court_frame, corners=upper_body))

In [ ]:
court_scene.ball.position + Offset(x=1, y=2)

Offset(x=3157, y=970)

In [25]:

ball = finder_service.findBall(frame=court_frame)
if ball is not None:
    show_frame(frame=court_scene.draw_ball(frame=court_frame))
else:
    print("No balls found")
